# Fine-Tune a GPT-4o-Mini Model to Answer Questions about Hurricanes

## Data Curation and Preparation

To demonstrate the fine-tuning process, we'll be working with a real-world dataset—recent updates to hurricane data from Wikipedia. The steps are as follows:

1. Collect Data: Gather the latest revisions from selected Wikipedia pages.
2. Generate Q&A Pairs: Turn the raw hurricane data into a useful set of question-and-answer pairs.
3. Create Fine-Tuning Dataset: Format the dataset to fit OpenAI's requirements for fine-tuning.


### Pull Wikipedia Revisions for Hurricane Topics (after a certain date)

In [1]:
# List of relevant topics
topics = [
    "List_of_United_States_hurricanes",
    "2024_Atlantic_hurricane_season",
    "Hurricane_Milton",
    "Hurricane_Beryl",
    "Hurricane_Francine",
    "Hurricane_Helene",
    "Hurricane_Isaac"
]

In [4]:
import requests
import difflib
import re

def get_wikipedia_revisions(article_title, start_date):
    """Fetches Wikipedia revisions after a certain date for a given article and collects the data into a list of dictionaries."""
    url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "prop": "revisions",
        "titles": article_title,
        "rvstart": start_date,
        "rvdir": "newer",  # Fetch revisions newer than start_date
        "rvlimit": "500",
        "rvprop": "timestamp|user|comment|content",
        "format": "json"
    }
    
    dataset = []  # List to hold the dictionaries
    continue_token = True  # To handle pagination
    
    while continue_token:
        response = requests.get(url, params=params)
        data = response.json()

        if 'query' in data and 'pages' in data['query']:
            pages = data['query']['pages']
            for page_id in pages:
                revisions = pages[page_id].get('revisions', [])
                if revisions:
                    previous_content = ""  # Initialize variable to hold previous revision content
                    for rev in revisions:
                        current_content = rev.get('*', '')  # Get the full revision content
                        timestamp = rev.get('timestamp', 'No timestamp')
                        user = rev.get('user', 'Anonymous')
                        comment = rev.get('comment', 'No comment')

                        # If there is a previous revision, calculate the difference (new content added)
                        if previous_content:
                            diff = list(difflib.unified_diff(previous_content.splitlines(), current_content.splitlines()))
                            new_content = "\n".join(line[1:] for line in diff if line.startswith('+') and not line.startswith('+++'))

                            # Extract citation links (URLs and <ref> tags)
                            citations = re.findall(r'<ref.*?>.*?</ref>|https?://\S+', new_content)

                            # Create a dictionary to hold the relevant information
                            revision_data = {
                                "article_title": article_title,
                                "timestamp": timestamp,
                                "user": user,
                                "comment": comment,
                                "new_content": new_content,
                                # "citations": citations
                            }

                            # Append the dictionary to the dataset list
                            dataset.append(revision_data)

                        # Update previous content
                        previous_content = current_content
                else:
                    print(f"No revisions found for {article_title} after {start_date}.")

        else:
            print(f"Error fetching data for {article_title}.")

        # Check if pagination is needed (continue parameter)
        if 'continue' in data:
            continue_token = data['continue'].get('rvcontinue', False)
            params['rvcontinue'] = continue_token
        else:
            continue_token = False  # End loop if no more pages
    
    return dataset  # Return the dataset


def fetch_revisions_for_topics(topics, start_date):
    """Fetches revisions for all topics after a certain date and returns a combined dataset."""
    full_dataset = []  # List to hold data for all topics
    for topic in topics:
        try:
            print(f"Fetching revisions for {topic} starting from {start_date}...")
            topic_data = get_wikipedia_revisions(topic, start_date)
            full_dataset.extend(topic_data)  # Append the data for each topic to the full dataset
        except Exception as e:
            print(f"Error fetching revisions for {topic}: {str(e)}")
    
    return full_dataset  # Return the full dataset


# Specify the start date (ISO 8601 format)
start_date = "2023-09-01T00:00:00Z"

# Fetch the latest revisions for all topics and store them in a dataset
dataset = fetch_revisions_for_topics(topics, start_date)

Fetching revisions for List_of_United_States_hurricanes starting from 2023-09-01T00:00:00Z...
Fetching revisions for 2024_Atlantic_hurricane_season starting from 2023-09-01T00:00:00Z...
Fetching revisions for Hurricane_Milton starting from 2023-09-01T00:00:00Z...
Fetching revisions for Hurricane_Beryl starting from 2023-09-01T00:00:00Z...
Fetching revisions for Hurricane_Francine starting from 2023-09-01T00:00:00Z...
Fetching revisions for Hurricane_Helene starting from 2023-09-01T00:00:00Z...
Fetching revisions for Hurricane_Isaac starting from 2023-09-01T00:00:00Z...


In [32]:
print("Number of examples in dataset: ", len(dataset))

Number of examples in dataset:  9489


In [33]:
# Example: Print the first 10 examples
for data in dataset[:10]:
    print(data)

{'article_title': 'List_of_United_States_hurricanes', 'timestamp': '2023-09-07T18:42:52Z', 'user': '2603:6011:B200:AB2:4053:9549:C77A:8B0F', 'comment': '/* Louisiana */', 'new_content': '| bgcolor=#{{storm colour|3}}|{{center|Category&nbsp;5 hurricane}}'}
{'article_title': 'List_of_United_States_hurricanes', 'timestamp': '2023-09-07T18:43:59Z', 'user': '2603:6011:B200:AB2:4053:9549:C77A:8B0F', 'comment': '/* Louisiana */', 'new_content': '| bgcolor=#{{storm colour|5}}|{{center|Category&nbsp;5 hurricane}}'}
{'article_title': 'List_of_United_States_hurricanes', 'timestamp': '2023-09-07T23:47:34Z', 'user': 'Jay Jor', 'comment': '/* Louisiana */ While Katrina was Category 5 in the Gulf of Mexico, within the state of Louisiana it was Category 3.', 'new_content': '| [[Hurricane Katrina|Katrina]]<ref>{{cite web |title=Hurricane Katrina, New Orleans, Louisiana, USA |url=https://eros.usgs.gov/media-gallery/earthshot/hurricane-katrina-new-orleans-louisiana-usa |website=USGS |publisher=U.S. Dept.

## Generate Q&A Pairs

Now that we have our dataset, we need to generate a set of question-and-answer pairs. We'll use OpenAI's API to analyze the new content and generate a set of question-answer pairs.


In [29]:
from openai import OpenAI
from pydantic import BaseModel
from typing import List, Literal
import json

# Define the Pydantic model for the output format
class QAItem(BaseModel):
    prompt: str
    completion: str

class QADataset(BaseModel):
    dataset: List[QAItem]


def generate_qa_pairs_from_changes(new_content, article_title):
    """
    Query OpenAI to analyze the new content and generate a set of question-answer pairs.
    If substantial information changes are detected (such as new sections, significant updates, or meaningful additions of facts),
    the function returns a list of question-answer pairs in the specified JSON format.
    """

    client = OpenAI()

    # Create a query prompt to ask OpenAI to generate question-answer pairs based on the content
    prompt = f"""
    The following is newly added content to the Wikipedia article titled '{article_title}'.
    Analyze the content and generate a set of specific question-answer pairs based on the new facts, updates, or meaningful changes.
    Focus on creating general questions that a person might ask and answered them comprehensively with the content provided.
    Do not ask questions that directly reference the date of the revision or the specific article title. 
    If a hurricane is mentioned, it should be referred to by its full name.
    Ignore trivial changes such as typos or formatting.

    Example questions: 
    - List the hurricanes that hit the US in 2024.
    - What was the most recent hurricane to hit the US?
    - What was the name of the hurricane that hit Florida in 2024?
    - What was the category of hurricane Beryl?
    - What was the path of hurricane Milton?

    New Content:
    \"\"\"{new_content[:3000]}\"\"\"

    Please return a set of question-answer pairs in the form of a JSON array where each item is an object containing 
    'prompt' as the question and 'completion' as the direct answer from the content.
    """

    try:
        response = client.beta.chat.completions.parse(
            model="gpt-4o-2024-08-06",  # or "gpt-4o" if available
            messages=[
                {"role": "system", "content": "You are a helpful assistant that generates a set of specific question-answer pairs based on the new facts, updates, or meaningful changes Wikipedia articles."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=1000,
            temperature=0.7,
            response_format=QADataset
        )

        # Parse the response from a JSON string to a Python dictionary
        response_json = json.loads(response.choices[0].message.content)['dataset'][0]

        # Create a new dictionary with the parsed content and additional fields
        result = {
            "prompt": response_json['prompt'],
            "completion": response_json['completion'],
            "article_title": article_title,
            "new_content": new_content
        }

        return result

    except Exception as e:
        return f"Error in generating QA pairs: {str(e)}"

In [34]:
# Test the Q&A pair generation
a = generate_qa_pairs_from_changes(dataset[256]['new_content'], dataset[256]['article_title'])
print(a)

{'prompt': 'What does NOAA predict for the 2024 Atlantic hurricane season?', 'completion': 'NOAA predicts an above-normal 2024 Atlantic hurricane season.', 'article_title': '2024_Atlantic_hurricane_season', 'new_content': '|align="left"|NOAA\n|align="left"|May 23, 2024\n|17–25\n|8–13\n|4–7\n|<ref name="NOAA season forecast">{{cite web|date=2024-05-23|title=NOAA predicts above-normal 2024 Atlantic hurricane season|url=https://www.noaa.gov/news-release/noaa-predicts-above-normal-2024-atlantic-hurricane-season|access-date=2024-05-23|website=National Oceanic and Atmospheric Administration|language=en-US}}</reF>\n|-'}


We'll now run this function over our entire dataset and collect the results. Depending on the length of the dataset, this may take a while. For a very large dataset, it can be more cost effective to run this in batch mode. For more information, see the [OpenAI Batch API](https://platform.openai.com/docs/guides/batch/overview).

Note: We have decided to only process revisions longer than 1000 characters, as shorter revisions are less likely to contain significant changes or new information, and it reduces the size of the dataset we can create.



In [56]:
from tqdm import tqdm

MAX_DATASET_SIZE = 100
MIN_CONTENT_LENGTH = 1000

# Master list to collect all Q&A pairs
master_qa_list = []

for data in tqdm(dataset[:MAX_DATASET_SIZE], desc="Processing dataset"):
    # Skip if the content is too short
    if len(data['new_content']) < MIN_CONTENT_LENGTH:
        continue

    # Call the function to generate QA pairs
    try:
        qa_response = generate_qa_pairs_from_changes(data['new_content'], data['article_title'])
        if isinstance(qa_response, dict) and 'prompt' in qa_response and 'completion' in qa_response:
            master_qa_list.append(qa_response)
        else:
            print(f"Skipping invalid response for article: {data['article_title']}")
    except Exception as e:
        print(f"Exception encountered for article {data['article_title']}: {str(e)}")
        # Continue processing instead of breaking
        continue

Processing dataset: 100%|██████████| 100/100 [01:59<00:00,  1.19s/it]


In [57]:
print("Number of examples in master_qa_list: ", len(master_qa_list))
# Example: Print the first 10 examples
for data in master_qa_list[:10]:
    print(data)

Number of examples in master_qa_list:  23
{'prompt': 'Which Pacific hurricane is known to have produced hurricane-force winds in California?', 'completion': 'The 1858 San Diego hurricane is the only Pacific tropical cyclone known to have produced hurricane-force winds in California.', 'article_title': 'List_of_United_States_hurricanes', 'new_content': 'The [[1858 San Diego hurricane]] is the only [[Pacific hurricane|Pacific]] tropical cyclone known to have produced hurricane-force winds in [[California]]; it affected [[San Diego, California|San Diego]] on October 2, 1858, though its center remained just offshore. In the 20th&nbsp;century, only four tropical cyclones produced tropical storm force winds in the southwestern United States: a [[1939 California tropical storm|tropical storm]] in September 1939 in California, [[Hurricane Joanne (1972)|Hurricane Joanne]] in October 1972 in Arizona, [[Hurricane Kathleen (1976)|Hurricane Kathleen]] in September 1976 in Arizona and California, [[

## Review the QA pairs in Label Studio
We can now review the QA pairs in Label Studio to ensure they are of high quality.

In [58]:
def format_qa_list_for_label_studio(qa_list):
    """
    Format the Q&A list into the desired structure for Label Studio.
    
    :param qa_list: List of dictionaries containing 'prompt' and 'completion' keys
    :return: Formatted list of Q&A pairs
    """
    formatted_list = []
    for qa_pair in qa_list:
        formatted_qa = {
            "data": {
                "question": qa_pair['prompt'],
                "article_title": qa_pair['article_title'],
                "context": qa_pair['new_content']
            },
            "predictions": [
                {
                    "model_version": "v1",
                    "result": [
                        {
                            "from_name": "completion",
                            "to_name": "question",
                            "type": "textarea",
                            "value": {
                                "text": [qa_pair['completion']]
                            }
                        }
                    ]
                }
            ]
        }
        formatted_list.append(formatted_qa)
    return formatted_list

In [59]:
# Save the formatted version of master_qa_list
formatted_qa_list = format_qa_list_for_label_studio(master_qa_list)
with open('formatted_qa_list.json', 'w') as f:
    json.dump(formatted_qa_list, f, indent=2)


In [60]:
# Define the URL where Label Studio is accessible and the API key for your user account
LABEL_STUDIO_URL = 'http://localhost:8080'  # Your Label Studio instance URL
API_KEY = 'b7fc43a9abe38ddbebca580cd1a6fd03b3778e8a'  # Your Label Studio API key

# Import the SDK and the client module
from label_studio_sdk.client import LabelStudio

# Connect to the Label Studio API and check the connection
ls = LabelStudio(base_url=LABEL_STUDIO_URL, api_key=API_KEY)

In [62]:
label_config = """
<View>
  <Header value="Please read the question, article title, and context. Modify the completion if needed." />
  
  <Header value="Article Title" />
  <Text name="article_title" value="$article_title" />

  <Header value="Context" />
  <Text name="context" value="$context" />

  <Header value="Question" />
  <Text name="question" value="$question" />

  <Header value="Completion (Prediction)" />
  <TextArea name="completion" toName="question"
            editable="true" required="true" />
</View>
"""

# Create a new project
project = ls.projects.create(
    title='Hurricane Data Project',
    description='Label questions and completions about hurricanes with their respective contexts and titles.',
    label_config=label_config
)

In [65]:
from label_studio_sdk.label_interface.objects import PredictionValue

for qa_pair in master_qa_list:
    # Create task data
    task_data = {
        "data": {
            "question": qa_pair['prompt'],
            "article_title": qa_pair['article_title'],
            "context": qa_pair['new_content']
        }
    }

    # Create the task in Label Studio
    task = ls.tasks.create(project=project.id, **task_data)
    task_id = task.id
    print(f"Task created with ID: {task_id}")

    # Create prediction data
    prediction = PredictionValue(
        model_version="v1",
        result=[
            {
                "from_name": "completion",
                "to_name": "question",
                "type": "textarea",
                "value": {
                    "text": [qa_pair['completion']]
                }
            }
        ]
    )

    # Insert prediction into the task
    ls.predictions.create(task=task_id, **prediction.model_dump())
    print(f"Prediction added for task ID: {task_id}")

Task created with ID: 56601
Prediction added for task ID: 56601
Task created with ID: 56602
Prediction added for task ID: 56602
Task created with ID: 56603
Prediction added for task ID: 56603
Task created with ID: 56604
Prediction added for task ID: 56604
Task created with ID: 56605
Prediction added for task ID: 56605
Task created with ID: 56606
Prediction added for task ID: 56606
Task created with ID: 56607
Prediction added for task ID: 56607
Task created with ID: 56608
Prediction added for task ID: 56608
Task created with ID: 56609
Prediction added for task ID: 56609
Task created with ID: 56610
Prediction added for task ID: 56610
Task created with ID: 56611
Prediction added for task ID: 56611
Task created with ID: 56612
Prediction added for task ID: 56612
Task created with ID: 56613
Prediction added for task ID: 56613
Task created with ID: 56614
Prediction added for task ID: 56614
Task created with ID: 56615
Prediction added for task ID: 56615
Task created with ID: 56616
Prediction a

## Annotate the Dataset and Download the Results
Once we have annotated the dataset, we can download the results and convert them into the format required for OpenAI fine-tuning.

Note: If we just want to simply create annotations from the predictions, we can select all the Tasks in Label Studio and click the "Create Annotations From Predictions" button.

In Label Studio, Views are customizable tabs in the Data Manager where you can configure filters and specify how tasks and data appear within the project. Each view is associated with a project and can include various filtering criteria, ordering options, and other display preferences.

We can create a View to filter the Tasks that are annotated, so we only export the annotated Tasks.

In [67]:
from label_studio_sdk.data_manager import Filters, Column, Type, Operator

filters = Filters.create(Filters.AND, [
    Filters.item(
        Column.completed_at,
        Operator.EMPTY,
        Type.Boolean,
        Filters.value(False)
    )
])


In [68]:
view = ls.views.create(
    project=project.id,
    data={
        'title': 'Annotated Tasks',
        'filters': filters
    }
)
tab = ls.views.get(id=view.id)

Now we can extract the annotated QA pairs from the annotated tasks and format them into a simple prompt-completion pair.

In [70]:
def format_task_data(task):
    """
    Format a single task's data into {'prompt': prompt, 'completion': completion} structure.
    """
    formatted_data = []
    
    # Assuming the question is in task.data['question']
    prompt = task.data.get('question', '')
    
    # Check annotations first, then predictions
    if task.annotations:
        for annotation in task.annotations:
            for result in annotation.get('result', []):
                if result.get('from_name') == 'completion' and result.get('to_name') == 'question':
                    completion = result.get('value', {}).get('text', [''])[0]
                    formatted_data.append({'prompt': prompt, 'completion': completion})
    elif task.predictions:
        for prediction in task.predictions:
            for result in prediction.get('result', []):
                if result.get('from_name') == 'completion' and result.get('to_name') == 'question':
                    completion = result.get('value', {}).get('text', [''])[0]
                    formatted_data.append({'prompt': prompt, 'completion': completion})
    
    return formatted_data

In [73]:
# List to store all formatted data
all_formatted_data = []

# Fetch annotated tasks
annotated_tasks = ls.tasks.list(view=tab.id, fields='all')

# Process each task
for task in annotated_tasks:
    formatted_task_data = format_task_data(task)
    all_formatted_data.extend(formatted_task_data)


In [75]:
print("Number of examples in master_qa_list: ", len(all_formatted_data))
# Example: Print the first 10 examples
for data in all_formatted_data[:10]:
    print(data)

Number of examples in master_qa_list:  23
{'prompt': 'Which Pacific hurricane is known to have produced hurricane-force winds in California?', 'completion': 'The 1858 San Diego hurricane is the only Pacific tropical cyclone known to have produced hurricane-force winds in California.'}
{'prompt': 'What is the only Pacific tropical cyclone known to have produced hurricane-force winds in California?', 'completion': 'The 1858 San Diego hurricane is the only Pacific tropical cyclone known to have produced hurricane-force winds in California.'}
{'prompt': 'What is the only Pacific tropical cyclone known to have produced hurricane-force winds in California?', 'completion': 'The only Pacific tropical cyclone known to have produced hurricane-force winds in California is the 1858 San Diego hurricane.'}
{'prompt': 'What is the only Pacific tropical cyclone known to have produced hurricane-force winds in California?', 'completion': 'The 1858 San Diego hurricane is the only Pacific tropical cyclone

In [86]:
# Save to a JSON file
import json
with open('annotated_qa_pairs.json', 'w') as f:
    json.dump(all_formatted_data, f, indent=2)

## Create the Fine-Tuning Dataset
We will now convert the list of QA pairs into the format required for OpenAI fine-tuning and write it to a file.

In [76]:
# System message for all entries
system_message = {"role": "system", "content": "You are a helpful assistant that answers questions about current events in hurricanes. Provide detailed answers."}

# List to store the converted dataset
new_format_dataset = []

# Convert each prompt-completion pair to the new format
for entry in all_formatted_data:
    new_entry = {
        "messages": [
            system_message,
            {"role": "user", "content": entry['prompt']},
            {"role": "assistant", "content": entry['completion']}
        ]
    }
    new_format_dataset.append(new_entry)


In [77]:
new_format_dataset[0]

{'messages': [{'role': 'system',
   'content': 'You are a helpful assistant that answers questions about current events in hurricanes. Provide detailed answers.'},
  {'role': 'user',
   'content': 'Which Pacific hurricane is known to have produced hurricane-force winds in California?'},
  {'role': 'assistant',
   'content': 'The 1858 San Diego hurricane is the only Pacific tropical cyclone known to have produced hurricane-force winds in California.'}]}

In [78]:
def write_to_jsonl(qa_list, filename):
    with open(filename, 'w') as f:
        for qa_item in qa_list:
            # Convert each dictionary to JSON format and write to file
            f.write(json.dumps(qa_item) + '\n')

In [79]:
write_to_jsonl(new_format_dataset, 'qa_pairs_openai_wiki_hurricane_dataset_format.jsonl')


# Dataset Analysis and Cost Estimates

We will use this file to analyze the data and create a new file with the correct format. The analysis functions are pulled from [Data preperation and analysis for chat model fine-tuning](https://cookbook.openai.com/examples/chat_finetuning_data_prep).

In [80]:
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

In [81]:
data_path = "qa_pairs_openai_wiki_hurricane_dataset_format.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 23
First example:
{'role': 'system', 'content': 'You are a helpful assistant that answers questions about current events in hurricanes. Provide detailed answers.'}
{'role': 'user', 'content': 'Which Pacific hurricane is known to have produced hurricane-force winds in California?'}
{'role': 'assistant', 'content': 'The 1858 San Diego hurricane is the only Pacific tropical cyclone known to have produced hurricane-force winds in California.'}


In [82]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [83]:
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

In [84]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))
    
print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 16385 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 16,385 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 61, 83
mean / median: 74.17391304347827, 75.0
p5 / p95: 71.0, 76.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 15, 29
mean / median: 25.47826086956522, 24.0
p5 / p95: 22.2, 29.0

0 examples may be over the 16,385 token limit, they will be truncated during fine-tuning


In [85]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 16385

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

Dataset has ~1706 tokens that will be charged for during training
By default, you'll train for 4 epochs on this dataset
By default, you'll be charged for ~6824 tokens


## Upload the Dataset to OpenAI
Once we are happy with the cost and details of our dataset, we can upload it to OpenAI.

In [77]:
client = OpenAI()

client.files.create(
  file=open("qa_pairs_openai_wiki_hurricane_dataset_format.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-qBVnzhGrEHvEPvZg6ZwvTfpK', bytes=4301148, created_at=1728916096, filename='qa_pairs_openai_wiki_hurricane_dataset_format.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

## Create a Fine-Tuning Job

We can now create a fine-tuning job programmatically or via the [fine-tuning UI](https://platform.openai.com/finetune), using the file ID for the file we just uploaded.

In [78]:
client.fine_tuning.jobs.create(
  training_file="file-qBVnzhGrEHvEPvZg6ZwvTfpK", 
  model="gpt-4o-mini-2024-07-18",
  suffix="wiki-hurricane-2024"
)

FineTuningJob(id='ftjob-YSQajs0bsLVi0IQfSv6b5jbz', created_at=1728916171, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-19ZNwom3AVpfo00n76H0UeLL', result_files=[], seed=433405727, status='validating_files', trained_tokens=None, training_file='file-qBVnzhGrEHvEPvZg6ZwvTfpK', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix='wiki-hurricane-2024')

We can check the status of our fine-tuning job using the job ID.

In [6]:
ft_job = client.fine_tuning.jobs.retrieve("ftjob-YSQajs0bsLVi0IQfSv6b5jbz")
print("ft_job.fine_tuned_model: ", ft_job.fine_tuned_model)

ft_job.fine_tuned_model:  ft:gpt-4o-mini-2024-07-18:personal:wiki-hurricane-2024:AIGr7s2N


## Use our fine-tuned model

Once our fine-tuning job is complete, we can use the new model to generate answers to questions about hurricanes.

In [3]:
from openai import OpenAI
client = OpenAI()

completion = client.beta.chat.completions.parse(
  model="ft:gpt-4o-mini-2024-07-18:personal:wiki-hurricane-2024:AIGr7s2N",
  messages=[
    {"role": "system", "content": "You are a helpful assistant that answers questions about current events in hurricanes. Provide detailed answers."},
    {"role": "user", "content": "What's the most recent hurricane to hit the US in 2024?"}
  ]
)
print(completion.choices[0].message)

ParsedChatCompletionMessage[NoneType](content='The most recent hurricane to hit the US in 2024 is Hurricane Milton, which struck Florida on October 9.', refusal=None, role='assistant', function_call=None, tool_calls=[], parsed=None)
